In [2]:
import time
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from sklearn_extra.cluster import KMedoids

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.mixture import GaussianMixture

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import estimate_bandwidth
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.cluster import MeanShift
from sklearn.cluster import AgglomerativeClustering
from konlpy.tag import Komoran

import pandas_datareader as pdr
from datetime import datetime

from scipy.cluster.hierarchy import linkage 
from scipy.cluster.hierarchy import dendrogram 

from tslearn.clustering import TimeSeriesKMeans
from tslearn.metrics import dtw

import scipy.spatial.distance as ssd

In [ ]:
# 크롤링(종목코드, 종목명)

url = "https://finance.naver.com/sise/entryJongmok.naver?&page={}"
name_lists= []
code_lists= []
for j in range(1,21):
    
    r = requests.get(url.format(j), headers = {'User-agent':'Mozilla/5.0'})
    html = r.text
    page = BeautifulSoup(html, 'html.parser')
    x = page.select('td[class="ctg"]')

    for i in range(0,10):
        code = x[i].find("a")["href"][22:]
        code_lists.append(code)
        
        name = page.select('td[class="ctg"]')[i].a.text
        name_lists.append(name)
    time.sleep(2)


In [ ]:
df2 = pd.DataFrame({"name": name_lists, "code": code_lists})

In [ ]:
#크롤링(기업개요)

url = 'https://finance.naver.com/item/coinfo.naver?code={}'
x1_list = []
x2_list = []
x3_list = []
for i in code_lists:
    r = requests.get(url.format(i), headers = {'User-agent':'Mozilla/5.0'})
    html = r.text
    page = BeautifulSoup(html, 'html.parser')
    x1 = page.select('p')[0].text
    x1_list.append(x1)
    x2 = page.select('p')[1].text
    x2_list.append(x2)
    x3 = page.select('p')[2].text
    x3_list.append(x3)
df2['x1'] = x1_list
df2['x2'] = x2_list
df2['x3'] = x3_list
df2

In [ ]:
#기업개요 하나로 묶은 x 컬럼 생성
x = []
for i in range(0,200):
    x.append(df.iloc[i]["x1"]+" "+df.iloc[i]["x2"]+" "+df.iloc[i]["x3"])
df["x"] = x
df

In [ ]:
# 크롤링 (업종)
url = "https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}"
labels_kse = []
labels_fics = []

for j in range(0,200):
    
    r = requests.get(url.format(df.iloc[j]["code"]))
    html = r.text
    page = BeautifulSoup(html, 'html.parser')
    y = page.select('span')[4].text
    y2 = page.select('span')[7].text
    y = y.replace('\xa0',' ')
    y2 = y2.replace('\xa0',' ')
    y = y[9:]
    y2 = y2[6:]
    y,y2
    labels_kse.append(y)
    labels_fics.append(y2)
    time.sleep(0.5)
    

df["labels_kse"] = labels_kse
df["labels_fics"] = labels_fics

In [ ]:
# 크롤링 (PER)
url = "https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}"
per = []

for j in range(0,200):
    r = requests.get(url.format(df_origin.iloc[j]["code"]))
    html = r.text
    page = BeautifulSoup(html, 'html.parser')
    y = page.select('dd')[4].text
    try :
        per.append(float(y))
    except :
        y = None
        per.append(y)

df["PER"] = per


In [ ]:
# 크롤링 (PBR)
url = "https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}"
pbr = []

for j in range(0,200):
    r = requests.get(url.format(df_origin.iloc[j]["code"]))
    html = r.text
    page = BeautifulSoup(html, 'html.parser')
    y = page.select('dd')[10].text
    try :
        pbr.append(float(y))
    except :
        y = None
        pbr.append(y)

df["PBR"] = pbr

In [ ]:
# 크롤링 (Dividend)
url = "https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}"
pbr = []

for j in range(0,200):
    r = requests.get(url.format(df_origin.iloc[j]["code"]))
    html = r.text
    page = BeautifulSoup(html, 'html.parser')
    y = page.select('dd')[12].text[:-1]
    try :
        pbr.append(float(y)/100)
    except :
        y = None
        pbr.append(y)

df["Dividend"] = pbr

In [ ]:
# 크롤링 (EV/EBITDA)
url = "https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={}&target=finsum_more"
pbr = []

for j in range(0,200):
    r = requests.get(url.format(df_origin.iloc[j]["code"]), headers = {'User-agent':'Mozilla/5.0'})
    html = r.text
    page = BeautifulSoup(html, 'html.parser')
    y = page.select("td[class='num']")[11].text
    try :
        pbr.append(float(y))
    except :
        y = None
        pbr.append(y)

df["EV/EBITDA"] = pbr

In [5]:
# 크롤링 (PCR)
url = "https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd={}&target=finsum_more"
pbr = []

r = requests.get(url.format('005930'), headers = {'User-agent':'Mozilla/5.0'})
html = r.text
page = BeautifulSoup(html, 'html.parser')
y = page.select("td[class='num']")[10].text
y

'7.67'

In [ ]:
# 크롤링 (시가, 종가, 최고가, 최저가)
df_price = pd.DataFrame()

code = "{}.ks"
start = datetime(2020,10,22)
end = datetime(2021,10,22)

for i in range(0,200):
    df_price[df.iloc[i]["name"]] = pdr.DataReader(code.format(df.iloc[i]["code"]), 'yahoo', start, end)["Close"].copy()
    time.sleep(1)
df_price

#df_price.to_pickle('C:/Users/Dohyung/Desktop/데이터 마이닝/price.fkl')

In [ ]:
# price data 를 등락률로 변환
df_updown = pd.DataFrame(index=df_price.index, columns=df_price.columns)
for i in range (1,248):
    for j in range(0,200):
        try :
            gap = df_price.iloc[i][j]-df_price.iloc[i-1][j]
            df_updown.iloc[i][j] = gap/df_price.iloc[i-1][j]
        except:
            pass

In [17]:
import pandas as pd
df_price = pd.read_pickle('./price_normal.fkl')
df_price = df_price.transpose().copy()
df_price = df_price.dropna(axis = 0)
df_price

Date,2020-10-22,2020-10-23,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-11-02,2020-11-03,2020-11-04,...,2021-10-08,2021-10-12,2021-10-13,2021-10-14,2021-10-15,2021-10-18,2021-10-19,2021-10-20,2021-10-21,2021-10-22
삼성전자,-2.477094,-2.463069,-2.435020,-2.519168,-2.631364,-2.757586,-2.967955,-2.855758,-2.659414,-2.701487,...,-0.878290,-1.228905,-1.256954,-1.172807,-1.074634,-1.060610,-1.004512,-1.046585,-1.060610,-1.032561
SK하이닉스,-2.159880,-2.122838,-2.166054,-2.203095,-2.184574,-2.258658,-2.369784,-2.388305,-2.277179,-2.166054,...,-1.499300,-1.653641,-1.622773,-1.511647,-1.227659,-1.307917,-1.270875,-1.258528,-1.357306,-1.221486
NAVER,-1.418506,-1.542362,-1.637636,-1.599526,-1.313705,-1.313705,-1.475670,-1.561417,-1.609054,-1.313705,...,0.401226,0.086822,0.277370,0.525082,0.496500,0.553664,0.763267,0.763267,0.820431,0.763267
삼성바이오로직스,-2.306606,-2.606760,-2.688620,-2.620404,-2.593117,-1.774516,-1.979166,-1.760872,-1.747229,-1.283355,...,0.040050,0.094623,0.026406,0.435707,0.572141,0.422064,0.558497,0.654001,0.558497,0.722217
LG화학,-2.365395,-2.232747,-2.280983,-2.449808,-2.329218,-2.220688,-2.703045,-2.558338,-2.160394,-2.232747,...,-0.858031,-0.472145,-0.327438,0.154918,0.010211,-0.098319,-0.050083,0.058447,-0.351556,-0.206849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
현대두산인프라코어,0.041122,-0.006779,-0.026502,-0.032137,-0.015232,-0.060314,-0.091308,-0.068767,-0.043408,-0.080038,...,-1.624592,-1.624592,-1.624592,-1.624592,-1.624592,-1.624592,-1.624592,-1.624592,-1.624592,-1.624592
롯데하이마트,-0.856272,-0.780919,-1.069770,-0.994417,-0.944183,-1.207916,-1.182798,-1.132563,-1.157681,-1.182798,...,-1.747940,-1.911204,-1.773058,-1.873528,-1.697706,-1.660029,-1.660029,-1.773058,-1.773058,-1.735382
LX하우시스,-1.494259,-1.520963,-1.583271,-1.752395,-1.405247,-1.583271,-1.601074,-1.823605,-1.743494,-1.725691,...,-0.746555,-1.040296,-0.880074,-0.853370,-0.773259,-0.995790,-0.880074,-0.942382,-0.986889,-1.049197
삼양식품,1.275320,1.210192,0.754294,0.933396,1.014807,1.079935,0.770576,0.705447,0.721729,1.682372,...,-1.411223,-1.720582,-1.622890,-1.557762,-1.427505,-1.476351,-1.525197,-1.639172,-1.606608,-1.671736


In [18]:
from dtaidistance import dtw
dtw_matrix = dtw.distance_matrix(df_price.to_numpy(), show_progress=True)

100%|████████████████████████████████████████████████████████████████████████████████| 194/194 [32:03<00:00,  9.91s/it]


In [19]:
dtw_matrix
df_dtw_matrix = pd.DataFrame(dtw_matrix)
df_dtw_matrix.index = df_price.index


In [20]:
df_dtw_matrix

,0,1,2,3,4,5,6,7,8,9,...,184,185,186,187,188,189,190,191,192,193
삼성전자,0.000000,2.869467,7.750365,7.998594,5.084879,7.080470,7.049312,3.760821,5.806505,6.108392,...,9.234514,5.142929,7.526803,7.147105,5.187178,14.580965,7.027403,5.022545,17.638922,22.867415
SK하이닉스,2.869467,0.000000,9.453427,9.052383,5.259753,8.613628,8.787274,4.788209,7.697084,5.142949,...,7.042576,4.740617,9.587700,5.740314,6.541818,13.296300,5.085923,4.241785,14.977939,20.482347
NAVER,7.750365,9.453427,0.000000,6.312767,8.391875,2.224759,4.463978,7.487354,4.939854,11.708098,...,10.604207,4.457010,3.184299,9.459375,3.308220,16.957284,11.236154,6.089922,22.704672,26.548097
삼성바이오로직스,7.998594,9.052383,6.312767,0.000000,8.005945,5.164912,5.424367,7.537216,6.486340,10.885658,...,9.984731,7.310370,5.443423,8.188671,5.488283,13.382969,10.323901,6.261957,16.792976,21.889155
LG화학,5.084879,5.259753,8.391875,8.005945,0.000000,8.104657,7.148895,4.213296,5.919671,7.578024,...,7.776600,5.542517,6.915840,6.345079,6.712172,14.323839,7.966059,5.534552,15.226068,20.536643
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
현대두산인프라코어,14.580965,13.296300,16.957284,13.382969,14.323839,15.337753,17.271091,15.151025,17.777913,11.061849,...,9.239220,14.336278,16.487159,10.646334,15.524986,0.000000,9.857081,11.836621,13.191419,14.903657
롯데하이마트,7.027403,5.085923,11.236154,10.323901,7.966059,10.229490,11.657364,7.164353,10.661715,4.760187,...,6.166942,6.182018,11.411102,3.763640,8.627309,9.857081,0.000000,4.861798,13.180435,18.586290
LX하우시스,5.022545,4.241785,6.089922,6.261957,5.534552,5.441999,7.043210,5.253966,6.763026,5.406743,...,5.270118,4.173046,6.523484,5.108574,4.472748,11.836621,4.861798,0.000000,17.454249,22.516920
삼양식품,17.638922,14.977939,22.704672,16.792976,15.226068,21.894130,20.581945,17.924761,21.570816,8.817446,...,11.508948,19.577548,22.257496,15.260678,21.023643,13.191419,13.180435,17.454249,0.000000,5.740042


In [8]:
df_dart = pd.read_pickle('./dart_khaiii.fkl')

In [9]:
import re


for i in range(0,200):
    test = df_dart.iloc[i]['x']
    test = re.sub('[☞※+#/\?:^$@*\"※~&%ㆍ!》■○◆]',' ',test)
    test = test.replace('\xa0',' ')
    test = test.replace('\n',' ')
    test = test.replace('㈜','')
    df_dart.iloc[i]['x'] =test


KeyError: 'x'

In [12]:
df_dart.to_csv('./dart.csv', encoding= 'euc-kr')

In [ ]:
# 신규 상장 기업인듯
for company in df_price:
    if df_price[company].isnull().sum() != 0:
        print('{}, {}개'.format(company, df_price[company].isnull().sum()))

In [ ]:
for company in df_price:
    print(df_price[company].mean(), df_price[company].std())
    
# 평균 빼고 표준편차로 나누기
for company in df_price:
    df_price[company] = (df_price[company][:] - df_price[company].mean()) / df_price[company].std()

In [ ]:
# 마지막 가격 기준
for company in df_price:
    df_price[company] = df_price[company][:] / df_price[company][-1] * 100

In [ ]:
#PER, Dividend 범주형으로 바꾸기
A = df_finance["PER"].copy()

for i in range(0,198):
    if A[i] == 0:
        A[i] = 0
    elif A[i] > 0 and A[i]<=2.0:
        A[i] = 1
    elif A[i] > 2 and A[i]<=4.0:
        A[i] = 2
    elif A[i] >4.0:
        A[i] = 3
A
df_finance["PER"] = A
df_finance


In [ ]:
A = df_finance["Dividend"].copy()

for i in range(0,198):
    if A[i] == 0:
        A[i] = 0
    elif A[i] < 0 and A[i]>=-5.0:
        A[i] = 1
    elif A[i] < -5.0 and A[i]>=-6.0:
        A[i] = 2
    elif A[i] < -6.0:
        A[i] = 3
A
df_finance["Dividend"] = A
df_finance